# Quickstart

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import damuta as da


<Figure size 432x288 with 0 Axes>

Load COSMIC signatures provided in damuta

In [ ]:
da.load_cosmic_V3()
